## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


С точки зрения математики, item_item и user_user идентичны. Различаются "направлением" данных, что у нас представляет столбец в датафрейме - товар, либо пользователя. Другими словами, если единственным действием для преобразования одного алгоритма в другой, будет транспонирование исходной матрицы. 

Что до различия с точки зрения бизнеса, user_user будет хорошо отрабатывать для "постоянного" клиента, о котором уже собраны данные. Плохо - для нового. На покойном ныне Имхонете, судя по всему, применялся этот подход. item_item - наоборот, мало учитывает персонификацию, и предлагает товары на основе "похожести", "совместных покупок", новые товары по этой логике будут непопулярны.

ALS, в отличие от предыдущих двух алгоритмов, разбивает большую разреженную матрицу на две "узких", экономя место для хранения данных. Поиск минимума среднеквадратичной ошибки ведется по ним. Вернее, алгоритм "фиксирует" один из параметров (пользователей), и ищет оптимальные товары.

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


* Фактор "новизны". Например, T/t, где T - гиперпараметр, кол-во дней. t - кол-во дней с момента появления товара в магазине.
* Сезонность. Довольно глупо рекоммендовать лыжи в середине весны.
* кол-во покупок товаров "из той же категории" - для этого потребуется "подтянуть" данные о товарах, группировать по категориям и считать суммарное кол-во покупок. Но если пользователь купил уже пять квадрокоптеров, тема его явно "цепляет", и нужно для этого юзера всем коптерам установить высокий вес. Изначально же этого не видно - все куплены в единственном экземпляре, и никак не отличимы, например, от единственного фотоаппарата, купленного тем же пользователем.

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


1. Трафик. Своеобразная "цена" за сжатость данных.
2. Считает лишь факт взаимодействия пользователя с товаром. Некоторые элементы очень популярны, они получат неоправданно высокое "внимание" и кол-во выделенной памяти.
3. Минимизирует квадрат потерь - не всегда это оптимально.

4. Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

Алгоритм оперирует параметром "встречаемости". Например, молоко покупают очень многие люди. Но это как раз снижает "ценность" продукта для рекоммендации. Мы бы хотели делать это персонально, и сводить клиента с интересующими его товарами, а не просто "впаривать" то, что покупается по-определению.

Если мы разделим q(m) (пользователи, покупающие молоко) на общее кол-во пользователей N - получим "вероятность" товара быть случайно купленным.
Чем меньше это число, тем более "уникальный" товар, и он лучше характеризует пользователя. Чтобы получить зависимость "чем больше тем больше", перевернем дробь N/q(m).

Чтобы сгладить кривую, и не "возносить до небес" редкие товары, возьмем логарифм. $\lg{\frac{N}{q(m)}}$

Итоговая вероятность того, что пользователь купил несколько разных товаров, будет равна произведению вероятностей покупки каждого из них. Поскольку вероятности у нас под логарифмом, будет сумма логарифмов.
$$\sum_{i=1}^n\lg{\frac{N}{q(m_i)}}$$

Это упрощенная реализация IDF в bm_25_weight. Именно тот механизм, который помогает перевзвесить матрицу, с учетом "уникальности запросов". Если юзер покупает зеркалку Кодак и объективы к нему, то это - не случайность, тогда как хлеб и молоко - распространенная "пара", и потому менее ценная для предсказаний.  


## Практическая часть


In [1]:
import implicit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from implicit.evaluation import precision_at_k

import warnings
warnings.filterwarnings('ignore')

In [3]:
# import data
# utils functions like in webinar

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.GridSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!